In [1]:
import dimod
import networkx as nx
import dwave_networkx as dnx
from dwave.embedding import pegasus
import neal, dwave.system
import numpy as np

# Solve Natural Linear Program as QUBO with default Solver Advantage_system1.1 and complete graph embedding

Problem IDs: 
4158919e-3e18-47d9-a561-3a73a36bc5f2
53b92b12-5dac-4e5c-a09a-24c69679fbc4
4b9a7001-6a4b-4fd3-a25b-599c309c2e9f

In [2]:
A_int = np.array([[0,1,2,3,4,1],
              [2,1,4,1,0,2],
              [1,5,2,3,1,1]])
m, n = A_int.shape
vartype = dimod.Vartype.BINARY
c_int = np.array([1,2,1,-3,1,-1])
b = np.array([7,8,5])
print(A_int.dot(np.array([1,0,0,0,1,3])))

[7 8 5]


# Problem statement:
Given $A\in Z^{mxn}, b\in Z^{m}, c\in Z^{n}$ 

find $x\in N^n$ with:

$\min c^T x $ with constraint $Ax=b$ 



This is equivalent to solving QUBO:

$min_x c^Tx + x^T A^T A x - 2 b^TAx + b^Tb$

eventually we need to set weights on the penalty parts $x^T A^T A x - 2 b^TAx + b^Tb$

In [3]:
def get_binary_form(c,bits):
    c_bin = []
    for c_j in c:
        c_bin += [c_j * 2**i for i in range(0,bits)]
    return np.array(c_bin)

In [4]:
# represent each x_i as 3 bits to the power of 2
bits = 3
# Set Penalty for Constraints aprox solution
P = max(abs(c_int))*3
c = get_binary_form(c_int,bits)
print(c)
A = np.array([get_binary_form(A_int[i], bits) for i in range(0,m)])
print(A)

[  1   2   4   2   4   8   1   2   4  -3  -6 -12   1   2   4  -1  -2  -4]
[[ 0  0  0  1  2  4  2  4  8  3  6 12  4  8 16  1  2  4]
 [ 2  4  8  1  2  4  4  8 16  1  2  4  0  0  0  2  4  8]
 [ 1  2  4  5 10 20  2  4  8  3  6 12  1  2  4  1  2  4]]


In [5]:
# Set up linear and quadratic coefficients
L = c.T - P * 2 * np.matmul(b.T,A)
Q = P * np.matmul(A.T,A)
offset = P * b.T.dot(b)
bqm = dimod.as_bqm(L, Q, offset, vartype)

In [6]:
def construct_x(sample, bits):
    x=[]
    for k in range(0,len(sample.keys()),bits):
        x += [sum(sample[k+j]* 2**j for j in range(0,bits))]
    return np.array(x)

In [7]:
def testSamples(sampleset, bits):
    print("Minimum Energy Level found in:",sampleset.first)
    X = []
    for sample, energy,  in sampleset.data(fields=['sample', 'energy']):
        x = construct_x(sample,bits)
        if all(A_int.dot(x)[i] == b[i] for i in range(0,m)) :
            X += [(x,energy)]
    return X

In [24]:
# Define target Pegasus Graph

M = 16 # Defines number of qubits in Pegasus graph : 24M(M-1)
P_M = dnx.pegasus_graph(M, data = True) # allow the use of pegasus indizes

# Define source complete graph

a = len(bqm.variables)
K_a = nx.complete_graph(a)

# Embedding K_a into P_M
embedding = pegasus.find_clique_embedding(K_a, target_graph=P_M)

sampler = dwave.system.FixedEmbeddingComposite(dwave.system.DWaveSampler(), embedding = embedding)

In [25]:
sampler.child.properties['num_qubits']

5760

In [26]:
sampleset = sampler.sample(bqm,num_reads = 2000)

In [27]:
sampleset.to_pandas_dataframe()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,chain_break_fraction,energy,num_occurrences
0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,0,0.000000,-1.0,2
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0.000000,6.0,1
2,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0.000000,12.0,5
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,1,0,0.000000,21.0,1
4,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0.000000,24.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177,1,0,1,1,0,0,1,0,0,0,...,0,0,1,0,1,1,1,0.000000,7365.0,1
1178,0,1,0,1,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0.055556,7693.0,1
1179,0,0,0,1,0,0,0,1,0,0,...,0,0,1,0,1,0,0,0.055556,3266.0,1
1180,1,0,0,1,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0.055556,3524.0,1


In [28]:
solutions_real = testSamples(sampleset, bits)
solutions_unique = { str(x) : energy for x, energy in solutions_real}
for x in solutions_unique.keys() :
    print("Found solution {} with Energy Level {}".format(x,solutions_unique[x]))

Minimum Energy Level found in: Sample(sample={0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 1, 16: 1, 17: 0}, energy=-1.0, num_occurrences=2, chain_break_fraction=0.0)
Found solution [1 0 0 0 1 3] with Energy Level -1.0


# First Run:
Minimum Energy (-1) never occurred

Minimum Energy Level found in: Sample(sample={0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 1, 16: 0, 17: 0}, energy=2.0, num_occurrences=1, chain_break_fraction=0.0)
Found solution [1 0 1 0 1 1] with Energy Level 2.0

# Second Run: 
Minimum Energy (-1) never occurred

Minimum Energy Level found in: Sample(sample={0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 1, 16: 0, 17: 0}, energy=2.0, num_occurrences=1, chain_break_fraction=0.0)
Found solution [1 0 1 0 1 1] with Energy Level 2.0

# Third Run:
Only one solution found

Minimum Energy Level found in: Sample(sample={0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 1, 16: 1, 17: 0}, energy=-1.0, num_occurrences=2, chain_break_fraction=0.0)
Found solution [1 0 0 0 1 3] with Energy Level -1.0


# Try find clique embedding function with numbers

In [12]:
a = len(bqm.variables)
embedding = pegasus.find_clique_embedding(a,m = a)

sampler = dwave.system.FixedEmbeddingComposite(dwave.system.DWaveSampler(), embedding = embedding)

DisconnectedChainError: chain for 0 is not connected

Error occurs over and over ... Maybe must wait for better pegasus embedding algorithm...